In [1]:
# -*- coding: utf-8 -*-
"""
SpaceX Web Scraping (without API) for Google Colab
Target: Wikipedia's "List of Falcon 9 and Falcon Heavy launches"
"""

# 1. Install necessary libraries
!pip install requests beautifulsoup4 pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re # For regular expressions, if needed for parsing strings

# For saving to Google Drive (optional)
from google.colab import drive
from IPython.display import display # For better DataFrame display

def scrape_spacex_launch_data_from_wikipedia():
    """
    Web scrapes SpaceX Falcon 9 and Falcon Heavy launch data from Wikipedia.
    Note: This is highly dependent on Wikipedia's page structure and can break easily.
    Returns a pandas DataFrame.
    """
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
    print(f"Attempting to scrape data from: {url}")

    try:
        # Fetch the HTML content
        response = requests.get(url, timeout=10) # Add a timeout
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.text, 'html.parser')
        print("Successfully fetched Wikipedia page.")

        # Find the main table containing launch data
        # Wikipedia tables often have specific classes or captions.
        # We'll look for tables with 'wikitable' class.
        # There might be multiple tables; we need to find the right one.
        # The main launch table usually has a caption or is the largest.
        tables = soup.find_all('table', class_='wikitable')

        # Heuristic: The largest table or one with a specific caption.
        # Let's find the one that is likely the main launch list.
        # Look for the table that contains columns like 'Flight No.', 'Date', 'Launch site'
        target_table = None
        for table in tables:
            # Check if headers contain key terms to identify the main launch table
            headers = [th.get_text(strip=True) for th in table.find_all('th')]
            if 'Flight No.' in headers and 'Launch date (UTC)' in headers and 'Launch site' in headers:
                target_table = table
                break

        if not target_table:
            print("Could not find the main launch data table on the page.")
            return pd.DataFrame()

        print("Found the main launch data table. Parsing...")

        # Get table headers
        headers = [th.get_text(strip=True) for th in target_table.find_all('th')]

        # Prepare a list to hold processed launch records
        launch_records = []

        # Iterate through table rows (skip header row)
        rows = target_table.find_all('tr')[1:] # Skip the first row (headers)

        for i, row in enumerate(rows):
            cols = [td.get_text(strip=True) for td in row.find_all('td')]

            # Debugging: Print headers and first few cols to understand structure
            # if i == 0:
            #    print("Table Headers:", headers)
            #    print("First Row Cols:", cols)
            #    print("Number of Headers:", len(headers))
            #    print("Number of Cols in first row:", len(cols))

            # Ensure the number of columns matches (can be tricky with colspan/rowspan)
            if len(cols) < len(headers) - 1: # -1 because some tables have initial index column
                # print(f"Skipping row {i} due to mismatched column count: {len(cols)} vs {len(headers)}")
                continue # Skip rows that don't match expected structure (e.g., notes, empty rows)

            record = {}
            # Map Wikipedia columns to requested columns
            # This mapping needs to be adjusted if Wikipedia's table structure changes.

            # Example mapping based on a typical Wikipedia launch table:
            # Assume order: Flight No., Launch date (UTC), Vehicle, Launch site,
            # Payload, Orbit, Customer, Outcome, Landing outcome (core), Core (landing),
            # Payload (landing), Remarks/Notes

            # Simplified mapping for common columns directly from td index
            # This is fragile! A more robust way uses the 'headers' list to map.
            # Let's try to map by header text if available.

            # Create a dictionary of column index to header name
            header_map = {h: idx for idx, h in enumerate(headers)}

            # --- Direct Mappings from table ---
            record['Flight No.'] = cols[header_map.get('Flight No.', 0)] if 'Flight No.' in header_map else None
            record['FlightNumber'] = record['Flight No.'] # Same as Flight No.

            # Date and Time
            date_time_str = cols[header_map.get('Launch date (UTC)', 1)] if 'Launch date (UTC)' in header_map else None
            if date_time_str:
                # Example: "1 May 2024\n01:30"
                date_match = re.search(r'(\d{1,2}\s[A-Za-z]+\s\d{4})', date_time_str)
                time_match = re.search(r'(\d{2}:\d{2})', date_time_str)
                record['Date'] = date_match.group(1) if date_match else None
                record['Time'] = time_match.group(1) if time_match else None
            else:
                record['Date'] = None
                record['Time'] = None

            record['Launch site'] = cols[header_map.get('Launch site', 3)] if 'Launch site' in header_map else None
            record['LaunchSite'] = record['Launch site']

            record['Payload'] = cols[header_map.get('Payload', 4)] if 'Payload' in header_map else None
            record['Orbit'] = cols[header_map.get('Orbit', 5)] if 'Orbit' in header_map else None
            record['Customer'] = cols[header_map.get('Customer', 6)] if 'Customer' in header_map else None

            record['Outcome'] = cols[header_map.get('Outcome', 7)] if 'Outcome' in header_map else None
            record['Launchoutcome'] = record['Outcome']

            record['BoosterVersion'] = cols[header_map.get('Vehicle', 2)] if 'Vehicle' in header_map else None
            record['Version Booster'] = record['BoosterVersion']

            # Booster landing (from "Landing outcome (core)" or similar)
            landing_outcome_core = cols[header_map.get('Landing outcome (core)', 8)] if 'Landing outcome (core)' in header_map else None
            record['Booster landing'] = landing_outcome_core

            # --- Columns very difficult or impossible to get reliably from table ---
            record['PayloadMass'] = None # Often buried in notes or requires separate lookup
            record['Longitude'] = None # Not in table, requires separate lookup for launch sites
            record['Latitude'] = None  # Not in table, requires separate lookup for launch sites
            record['Flights'] = None # Hard to get core-specific flight count from main table
            record['GridFins'] = None # Not typically in this type of table
            record['Reused'] = None # Can sometimes infer from 'Core' column, but not explicit for all
            record['Legs'] = None # Not typically in this type of table
            record['LandingPad'] = None # Usually listed with landing outcome, but formal name may vary
            record['Block'] = None # Not typically in this type of table
            record['ReusedCount'] = None # Not directly available from main table
            record['Serial'] = None # Sometimes in 'Core' column, but not consistently parseable for all

            # Attempt to extract 'Reused' and 'Serial' from 'Core' column if it exists and structured
            # This is highly heuristic and may not work for all rows/table versions
            if 'Core' in header_map and len(cols) > header_map['Core']:
                core_info_str = cols[header_map['Core']]
                if '—' in core_info_str or 'new' in core_info_str.lower():
                    record['Reused'] = False
                elif 'Refurbished' in core_info_str or 'reused' in core_info_str.lower():
                    record['Reused'] = True

                # Try to extract serial (e.g., B1049.10)
                serial_match = re.search(r'(B\d{4}\.\d+)', core_info_str)
                if serial_match:
                    record['Serial'] = serial_match.group(1)
                else: # Try other patterns like just B10XX
                    serial_match = re.search(r'(B\d{4})', core_info_str)
                    if serial_match:
                        record['Serial'] = serial_match.group(1)

            launch_records.append(record)

            if (i + 1) % 50 == 0:
                print(f"Processed {i + 1} rows...")

        print(f"Finished parsing {len(launch_records)} launch records.")
        df = pd.DataFrame(launch_records)
        return df

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
    except Exception as e:
        print(f"An unexpected error occurred during parsing: {e}")

    return pd.DataFrame() # Return empty DataFrame on error

if __name__ == "__main__":
    spacex_df_scraped = scrape_spacex_launch_data_from_wikipedia()

    if not spacex_df_scraped.empty:
        print("\nSpaceX Scraped Launch Data (first 5 rows):")
        display(spacex_df_scraped.head()) # Use display() for better Colab output

        print(f"\nTotal launches scraped: {len(spacex_df_scraped)}")
        print("\nAll columns available from scraping:")
        print(spacex_df_scraped.columns.tolist())

        # --- Colab Specific: Saving the DataFrame ---

        # Option 1: Save to a CSV file in the Colab temporary storage
        csv_filename = "spacex_scraped_launches.csv"
        spacex_df_scraped.to_csv(csv_filename, index=False)
        print(f"\nData saved to {csv_filename} in Colab environment.")
        print(f"You can download this file from the 'Files' tab (left sidebar) or using: from google.colab import files; files.download('{csv_filename}')")

        # Option 2: Save to Google Drive (persistent storage)
        try:
            print("\nAttempting to mount Google Drive for persistent storage...")
            drive.mount('/content/drive', force_remount=True)
            drive_path = '/content/drive/MyDrive/spacex_scraped_data/' # Create this folder in your Drive
            import os
            os.makedirs(drive_path, exist_ok=True)
            drive_csv_path = os.path.join(drive_path, csv_filename)
            spacex_df_scraped.to_csv(drive_csv_path, index=False)
            print(f"Data saved to Google Drive at: {drive_csv_path}")
        except Exception as e:
            print(f"\nCould not mount Google Drive or save to Drive: {e}")
            print("To save to Drive, you need to authenticate your Google account (follow prompts).")
    else:
        print("Failed to retrieve SpaceX launch data via web scraping.")

Attempting to scrape data from: https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches
Successfully fetched Wikipedia page.
Could not find the main launch data table on the page.
Failed to retrieve SpaceX launch data via web scraping.


In [2]:
# -*- coding: utf-8 -*-
"""
SpaceX Web Scraping (without API) for Google Colab
Target: Wikipedia's "List of Falcon 9 and Falcon Heavy launches"
"""

# 1. Install necessary libraries
# !pip install requests beautifulsoup4 pandas # Already satisfied from output

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re # For regular expressions, if needed for parsing strings

# For saving to Google Drive (optional)
from google.colab import drive
from IPython.display import display # For better DataFrame display

def scrape_spacex_launch_data_from_wikipedia():
    """
    Web scrapes SpaceX Falcon 9 and Falcon Heavy launch data from Wikipedia.
    Note: This is highly dependent on Wikipedia's page structure and can break easily.
    Returns a pandas DataFrame.
    """
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
    print(f"Attempting to scrape data from: {url}")

    try:
        # Fetch the HTML content
        response = requests.get(url, timeout=10) # Add a timeout
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.text, 'html.parser')
        print("Successfully fetched Wikipedia page.")

        # Find the main table containing launch data
        # Wikipedia tables often have specific classes or captions.
        # We'll look for tables with 'wikitable' class.
        tables = soup.find_all('table', class_='wikitable')

        target_table = None
        # We need to find the table that has specific headers relevant to launches.
        # Let's be more flexible with header names that might have subtle differences
        # or additional elements (like <sup> tags for references).

        for table in tables:
            # Get headers, stripping any reference superscripts or extra spaces
            headers_raw = [th.get_text(strip=True) for th in table.find_all('th')]
            # Clean headers by removing content within brackets, e.g., '[n]' for references
            headers_cleaned = [re.sub(r'\[.*?\]', '', h).strip() for h in headers_raw]

            # Check for core identifying headers
            if 'Flight No.' in headers_cleaned and \
               'Launch date (UTC)' in headers_cleaned and \
               'Launch site' in headers_cleaned and \
               'Outcome' in headers_cleaned:
                target_table = table
                headers = headers_cleaned # Use the cleaned headers for mapping
                print("Identified table by its core headers.")
                break

        if not target_table:
            # Fallback heuristic: Sometimes the most relevant table is the largest.
            # Or, it might have a specific caption.
            # Let's try to find tables with specific captions or a combination of elements.
            # Or, loop through headers and print them to debug exactly what's being found.
            print("Could not find the main launch data table using core headers. Trying alternative identification...")

            # This is a common way to identify the *first* main table
            # after the summary/index tables.
            # A common approach is to find the table with "Flight No." as a header.
            for table in tables:
                headers_raw = [th.get_text(strip=True) for th in table.find_all('th')]
                headers_cleaned = [re.sub(r'\[.*?\]', '', h).strip() for h in headers_raw]

                if 'Flight No.' in headers_cleaned:
                    target_table = table
                    headers = headers_cleaned
                    print("Identified table by 'Flight No.' header as a fallback.")
                    break

            if not target_table:
                print("Still could not find the main launch data table on the page.")
                return pd.DataFrame()

        print("Found the main launch data table. Parsing...")

        # At this point, 'headers' should contain the cleaned headers of 'target_table'

        # Prepare a list to hold processed launch records
        launch_records = []

        # Iterate through table rows (skip header row)
        rows = target_table.find_all('tr')[1:] # Skip the first row (headers)

        for i, row in enumerate(rows):
            cols = [td.get_text(strip=True) for td in row.find_all('td')]

            # Handle rows that might be sub-headers or notes, by checking if they are too short
            # A common issue is colspan/rowspan which makes len(cols) inconsistent.
            # We'll skip rows that have significantly fewer columns than expected based on headers.
            # A good minimum might be 5-6 core columns (Flight No, Date, Site, Payload, Outcome).
            if len(cols) < 5:
                continue

            record = {}

            # Create a dictionary of column header to its index for robust mapping
            header_map = {h: idx for idx, h in enumerate(headers)}

            # --- Direct Mappings from table using header_map ---
            record['Flight No.'] = cols[header_map.get('Flight No.')] if 'Flight No.' in header_map and header_map.get('Flight No.') < len(cols) else None
            record['FlightNumber'] = record['Flight No.']

            # Date and Time
            date_time_str = cols[header_map.get('Launch date (UTC)')] if 'Launch date (UTC)' in header_map and header_map.get('Launch date (UTC)') < len(cols) else None
            if date_time_str:
                # Remove reference brackets like [a] or [b]
                date_time_str = re.sub(r'\[.*?\]', '', date_time_str).strip()
                date_match = re.search(r'(\d{1,2}\s[A-Za-z]+\s\d{4})', date_time_str)
                time_match = re.search(r'(\d{2}:\d{2})', date_time_str)
                record['Date'] = date_match.group(1) if date_match else None
                record['Time'] = time_match.group(1) if time_match else None
            else:
                record['Date'] = None
                record['Time'] = None

            record['Launch site'] = cols[header_map.get('Launch site')] if 'Launch site' in header_map and header_map.get('Launch site') < len(cols) else None
            record['LaunchSite'] = record['Launch site']

            record['Payload'] = cols[header_map.get('Payload')] if 'Payload' in header_map and header_map.get('Payload') < len(cols) else None
            record['Orbit'] = cols[header_map.get('Orbit')] if 'Orbit' in header_map and header_map.get('Orbit') < len(cols) else None
            record['Customer'] = cols[header_map.get('Customer')] if 'Customer' in header_map and header_map.get('Customer') < len(cols) else None

            record['Outcome'] = cols[header_map.get('Outcome')] if 'Outcome' in header_map and header_map.get('Outcome') < len(cols) else None
            record['Launchoutcome'] = record['Outcome']

            record['BoosterVersion'] = cols[header_map.get('Vehicle')] if 'Vehicle' in header_map and header_map.get('Vehicle') < len(cols) else None
            record['Version Booster'] = record['BoosterVersion']

            # Booster landing (from "Landing outcome (core)" or similar)
            landing_outcome_core = cols[header_map.get('Landing outcome (core)')] if 'Landing outcome (core)' in header_map and header_map.get('Landing outcome (core)') < len(cols) else None
            record['Booster landing'] = landing_outcome_core

            # --- Columns very difficult or impossible to get reliably from table ---
            record['PayloadMass'] = None
            record['Longitude'] = None
            record['Latitude'] = None
            record['Flights'] = None
            record['GridFins'] = None
            record['Reused'] = None
            record['Legs'] = None
            record['LandingPad'] = None
            record['Block'] = None
            record['ReusedCount'] = None
            record['Serial'] = None

            # Attempt to extract 'Reused' and 'Serial' from 'Core' column if it exists and structured
            if 'Core' in header_map and header_map.get('Core') < len(cols):
                core_info_str = cols[header_map['Core']]
                if '—' in core_info_str or 'new' in core_info_str.lower():
                    record['Reused'] = False
                elif 'Refurbished' in core_info_str or 'reused' in core_info_str.lower():
                    record['Reused'] = True

                # Try to extract serial (e.g., B1049.10)
                serial_match = re.search(r'(B\d{4}(?:\.\d+)?)', core_info_str) # Made regex more flexible for .XX or no .
                if serial_match:
                    record['Serial'] = serial_match.group(1)

            launch_records.append(record)

            if (i + 1) % 50 == 0:
                print(f"Processed {i + 1} rows...")

        print(f"Finished parsing {len(launch_records)} launch records.")
        df = pd.DataFrame(launch_records)
        return df

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
    except Exception as e:
        print(f"An unexpected error occurred during parsing: {e}")

    return pd.DataFrame() # Return empty DataFrame on error

if __name__ == "__main__":
    spacex_df_scraped = scrape_spacex_launch_data_from_wikipedia()

    if not spacex_df_scraped.empty:
        print("\nSpaceX Scraped Launch Data (first 5 rows):")
        display(spacex_df_scraped.head())

        print(f"\nTotal launches scraped: {len(spacex_df_scraped)}")
        print("\nAll columns available from scraping:")
        print(spacex_df_scraped.columns.tolist())

        # --- Colab Specific: Saving the DataFrame ---
        csv_filename = "spacex_scraped_launches.csv"
        spacex_df_scraped.to_csv(csv_filename, index=False)
        print(f"\nData saved to {csv_filename} in Colab environment.")
        print(f"You can download this file from the 'Files' tab (left sidebar) or using: from google.colab import files; files.download('{csv_filename}')")

        try:
            print("\nAttempting to mount Google Drive for persistent storage...")
            drive.mount('/content/drive', force_remount=True)
            drive_path = '/content/drive/MyDrive/spacex_scraped_data/'
            import os
            os.makedirs(drive_path, exist_ok=True)
            drive_csv_path = os.path.join(drive_path, csv_filename)
            spacex_df_scraped.to_csv(drive_csv_path, index=False)
            print(f"Data saved to Google Drive at: {drive_csv_path}")
        except Exception as e:
            print(f"\nCould not mount Google Drive or save to Drive: {e}")
            print("To save to Drive, you need to authenticate your Google account (follow prompts).")
    else:
        print("Failed to retrieve SpaceX launch data via web scraping.")

Attempting to scrape data from: https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches
Successfully fetched Wikipedia page.
Could not find the main launch data table using core headers. Trying alternative identification...
Identified table by 'Flight No.' header as a fallback.
Found the main launch data table. Parsing...
Processed 50 rows...
Processed 100 rows...
Processed 150 rows...
Finished parsing 98 launch records.

SpaceX Scraped Launch Data (first 5 rows):


,Flight No.,FlightNumber,Date,Time,Launch site,LaunchSite,Payload,Orbit,Customer,Outcome,...,Longitude,Latitude,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial
0,"January 3, 202314:56[17]","January 3, 202314:56[17]",None,None,None,None,Unknown[j],Various,Success,None,...,None,None,None,None,None,None,None,None,None,None
1,"January 10, 202304:50[23]","January 10, 202304:50[23]",None,None,None,None,"6,000 kg (13,000 lb)",OneWeb,Success,None,...,None,None,None,None,None,None,None,None,None,None
2,"January 15, 202322:56[29]","January 15, 202322:56[29]",None,None,None,None,"~3,750 kg (8,270 lb)",USSF,Success,None,...,None,None,None,None,None,None,None,None,None,None
3,"January 18, 202312:24[33]","January 18, 202312:24[33]",None,None,None,None,"4,352 kg (9,595 lb)",USSF,Success,None,...,None,None,None,None,None,None,None,None,None,None
4,"January 19, 202315:43[39]","January 19, 202315:43[39]",None,None,None,None,"15,000 kg (33,000 lb)",SpaceX,Success,None,...,None,None,None,None,None,None,None,None,None,None



Total launches scraped: 98

All columns available from scraping:
['Flight No.', 'FlightNumber', 'Date', 'Time', 'Launch site', 'LaunchSite', 'Payload', 'Orbit', 'Customer', 'Outcome', 'Launchoutcome', 'BoosterVersion', 'Version Booster', 'Booster landing', 'PayloadMass', 'Longitude', 'Latitude', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial']

Data saved to spacex_scraped_launches.csv in Colab environment.
You can download this file from the 'Files' tab (left sidebar) or using: from google.colab import files; files.download('spacex_scraped_launches.csv')

Attempting to mount Google Drive for persistent storage...
Mounted at /content/drive
Data saved to Google Drive at: /content/drive/MyDrive/spacex_scraped_data/spacex_scraped_launches.csv
